The complete record of changes in the dataset/model architecture/learning techniques used can be seen at https://docs.google.com/document/d/1eF0cUl0fvmg4Zk7Ej1xws9iGObedpPTzDhYCx79zp9I 

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Flatten, Dense, GaussianNoise, Dropout, RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.models import load_model
from tensorflow_addons.optimizers import MultiOptimizer

import os
import random
from joblib import load, dump
import gc
import sys

tf.keras.backend.set_floatx('float32')
input_shape = (128, 128, 3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
base_model = MobileNetV2(include_top=False, input_shape=input_shape)

In [ ]:
train_layers = ["Conv_1", "block_16_project", "block_16_depthwise", "block_16_expand",
                "block_15_project", "block_15_depthwise", "block_15_expand",
                "block_14_project", "block_14_depthwise", "block_14_expand"
                ]

for layer in base_model.layers:
    layer.trainable = layer.name in train_layers
    if layer.name in train_layers:
        print(layer.name)

block_14_expand
block_14_depthwise
block_14_project
block_15_expand
block_15_depthwise
block_15_project
block_16_expand
block_16_depthwise
block_16_project
Conv_1


In [ ]:
def mat_shuffle(matrixs):
    widths = np.cumsum([0]+[matrix.shape[1] for matrix in matrixs])
    big = np.hstack(matrixs)
    np.random.shuffle(big)
    return [big[:,widths[i]:widths[i+1]].copy() for i in range(len(matrixs))]

In [ ]:
input_x = Input(shape=input_shape)

# augmentation layers
x = RandomFlip("horizontal")(input_x)
x = GaussianNoise(0.03)(x)
x = RandomRotation(0.3)(x)
x = RandomContrast(0.3)(x)

x = base_model(x)

x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
output_layer = Dense(1)(x)

model = Model(input_x, output_layer)

base_layers = [layer for layer in base_model.layers if layer.name in train_layers]
added_layers = [model.layers[-1], model.layers[-3]]

optimizer = MultiOptimizer([(Adam(1e-5), base_layers), (Adam(1e-3), added_layers)])

model.compile(optimizer=optimizer, loss='logcosh')

In [ ]:
results = dict()
c = 0
while True:
    for i in range(2):
        images = load(f"data/{i}_images.joblib")
        ages = load(f"data/{i}_ages.joblib")

        results[f"{c}_{i}"] = model.fit(images, ages[:, None], epochs=128, batch_size=256,
                              validation_split=0.2, use_multiprocessing=True, verbose=2)
        
        val = (len(ages)*4)//5
        pred = model.predict(images[val:]).flatten()

        plt.scatter(pred, ages[val:], alpha=0.1, lw=0)
        plt.scatter(np.arange(min(pred),max(pred), 0.1),
                    np.arange(min(pred),max(pred), 0.1),
                    color='red', alpha=0.1, lw=0)
        plt.show()
        
        del images
        gc.collect()
        del ages
        gc.collect()
        del pred
        gc.collect()
        
        model.save(f'models/{c}_model_v3.h5')
    c += 1

In [ ]:
def predict(url):
    image = tf.image.resize(tf.keras.utils.img_to_array(tf.keras.utils.load_img(url)), input_shape[:2])
    image = (image.numpy().astype('float32')-127.5)/127.5
    image = image[None, ...]
    return float(model.predict(image)*15.54)+35.3